# Color Matching Experiment - Student Interface


###This notebook will help you submit your experiment to the Opentrons OT-2 liquid handler robot stationed at the Acceleration Consortium (AC) Training Lab. The robot is set up with remote access and a wireless charging color sensor to perform the liquid color matching experiment.

<img width="300" alt="Image" src="https://github.com/user-attachments/assets/6e31d0e7-5331-4382-a240-e85b7c184ff5" />

---

A single submission process consists of:
1. Verify your student ID
2. Enter your experiment parameters (Red, Yellow, Blue volumes)
3. Submit experiment and receive results

Input: Student_id, R, Y and B color volumes

Output: 8-channel color sensor intensity results








---


For a full-cycle color matching experiment:

- You will have a **target color mixture** (a specific combination of R, Y, and B volumes) and the corresponding **target 8-channel color sensor readings**.  

- The experiment begins by submitting an initial set of **R, Y, and B values**, which will be mixed by OT-2 and measured by the sensor. You will then compare the **current 8-channel sensor readings** with the **target readings**.  

- Using an **optimization algorithm**, you will iteratively adjust the R, Y, and B values, refining your guesses to get closer to the target. This process continues until the measured sensor data matches the **target color mixture**.  








---



Let' s start by setting up the Gradio API and connecting it to the OT-2 Liquid Color Matching Experiment Queue on the Hugging Face Space.





In [ ]:
!pip install gradio_client
from gradio_client import Client
import time
import json
import time

# Initialize client - replace with your space URL
client = Client("https://accelerationconsortium-ot-2-lcm.hf.space/")

def verify_student_id(student_id):
    """Verify student ID using Gradio client"""
    try:
        result = client.predict(student_id, api_name="/verify_student_id")
        return result
    except Exception as e:
        print(f"Error during verification: {str(e)}")
        return None

def queue_status():
    try:
        queue_status = client.predict(
            api_name="/update_queue_display"
        )
        print(queue_status)

    except Exception as e:
            print(f"Error getting queue status: {str(e)}")

def validate_colors(r_vol,y_vol,b_vol):
    total_volume = r_volume + y_volume + b_volume
    if any(v < 1 or v > 300 for v in [r_volume, y_volume, b_volume]):
        print("Error: Each volume must be between 1 and 300 µL")
    elif total_volume > 300:
        print(f"Error: Total volume ({total_volume}µL) exceeds 300µL limit")
    else:
        print(f"Total volume: {total_volume}µL - Valid!")


def submit_experiment(student_id, r_vol, y_vol, b_vol, info=True):

    try:
        if info:
          print("Waiting results...")

        api_endpoint = "/debug" if student_id == "debug" else "/submit"
        job = client.submit(
        student_id,
        r_vol,
        y_vol,
        b_vol,
        api_name = api_endpoint
       )

        result = job.result()
        if info:
          print(json.dumps(result, indent=2))

        return result

    except Exception as e:
        print(f"Error during submission: {str(e)}")
        return None


print("Setup complete!")



## Step 1: Verify Your Student ID

You can see your remaining credit from the output

In [ ]:
student_id = input("Enter your student ID: ")
result = verify_student_id(student_id)
print("\nVerification result:", result)

## Step 2: Enter Color Volumes

Remember:
- Each volume must be between 1-300 µL
- Total volume cannot exceed 300 µL

In [ ]:
try:
    r_volume = float(input("Red volume (µL): "))
    y_volume = float(input("Yellow volume (µL): "))
    b_volume = float(input("Blue volume (µL): "))

    validate_colors(r_volume,y_volume,b_volume)

except ValueError:
    print("Error: Please enter valid numbers")

## Step 3: Submit Your Experiment

- Obtain queue status
- Submit your experiment with ID and colors volumes

In [ ]:
queue_status()

In [ ]:
try:
    if r_volume + y_volume + b_volume <= 300 and all(1 <= v <= 300 for v in [r_volume, y_volume, b_volume]):
        print("Submitting experiment...")
        submit_experiment(student_id, r_volume, y_volume, b_volume)
    else:
        print("Please fix the volume errors before submitting")
except NameError:
    print("Please complete the previous steps first")

## Congrats and what's next?

You have finished your first color mixing and obtained the first eight channels data readout from the color sensor.

Now you can either prepare a series of R, Y, and B values and use ```result = submit_experiment(student_id, r_vol, y_vol, b_vol)``` to obtain more data, or add your optimization algorithm to estimate the next R, Y, and B values from previous results between each iteration.

You can use ```student_id = "debug"``` to test your code without submitting an actual experiment to the OT-2. By enabling debug feature, you will obtain a dummy sensor result in 10 seconds.

When you think your code is ready, use your actual student_id to submit the experiment.




In [ ]:
"""
student_id = "debug"
debug_result = submit_experiment(student_id, 10, 10, 10)
"""

In [ ]:
#student_id = "ansarir5"

## Collect your "data lake"
For this practical, each of you will have 30 quota to build your own data cube. Please consider to use what we learned about statistical design of experiments to effecitvely cover the searching space.Of course, we don't want to call the API one by one. So let's batch-submit our tasks by creating a CSV file. This way, your input parameters will be properly stored and organized.

In [ ]:
# firstly let's try to load the created csv file
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
color_parameters = pd.read_csv('drive/MyDrive/Assignments/MSE1003/Assignment 2/colors3.csv')

In [ ]:
def validate_colors(r, y, b):
    """Check if volumes are within the valid range (1-300 µL)."""
    if not all(1 <= v <= 300 for v in [r, y, b]):
        return "Volumes must be between 1 and 300 µL."
    if r + y + b > 300:
        return "Total volume exceeds the 300 µL limit."
    return None  # No issues


In [ ]:
# lets fake a invalid data by changing a value in color_parameters and see what will happen

# color_parameters['B'][0] += 2

In [ ]:
# Step 1: Examination phase
issues = []  # Store rows with potential concerns
valid_rows = []  # Store valid rows

for index, row in color_parameters.iterrows():
    try:
        r_volume = float(row["R"])
        y_volume = float(row["Y"])
        b_volume = float(row["B"])

        issue = validate_colors(r_volume, y_volume, b_volume)
        if issue:
            issues.append((index, row, issue))
        else:
            valid_rows.append((index, r_volume, y_volume, b_volume))

    except ValueError:
        issues.append((index, row, "Invalid data format"))

In [ ]:
# Step 2: Inform the user about concerns
if issues:
    print("\n⚠️ Some rows have issues:")
    for index, row, issue in issues:
        print(f"Row {index}: {issue} - {row.to_dict()}")

    proceed = input("\nDo you want to proceed with submission? (yes/no): ").strip().lower()
    if proceed != "yes":
        print("Submission aborted.")
        exit()

In [ ]:
# Step 3: Submit valid experiments and store results
results = []
total_tasks = len(valid_rows)

for count, (index, r_volume, y_volume, b_volume) in enumerate(valid_rows, start=1):
    print(f"{count}/{total_tasks} submitted...")
   # response = submit_experiment("test", r_volume, y_volume, b_volume, info=False)  # Replace "student_id" as needed
    response = submit_experiment("ansarir5", r_volume, y_volume, b_volume, info=False)  # Replace "student_id" as needed
    sensor_data = response["Sensor Data"]

    # Store R, Y, B along with sensor readings
    results.append({
        "Red": r_volume, "Yellow": y_volume, "Blue": b_volume,
        **sensor_data  # Unpacking sensor data into columns
    })

    # Show progress update
    print(f"{count}/{total_tasks} completed...")


In [ ]:
# Step 4: Save results to CSV
output_filename = "color_results.csv"
df_results = pd.DataFrame(results)
df_results.to_csv(output_filename, index=False)

print(f"\n✅ Experiment results saved to {output_filename} successfully!")

In [ ]:
# Step 5: you may want to have a email notification when it is done

import smtplib
import ssl
from email.message import EmailMessage

def send_email_notification(recipient_email, output_filename):
    sender_email = "mse403h@163.com"
    sender_password = "MQFUF4DM4eUaku6P"  # Use an App Password instead of your login password

    subject = "Experiment Results Completed"
    body = f"""Dear User,

All experiment tasks have been successfully completed. The results are attached.

Best,
Runze
"""

    # Create the email message
    msg = EmailMessage()
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg["Subject"] = subject
    msg.set_content(body)

    # Attach the CSV file
    with open(output_filename, "rb") as file:
        msg.add_attachment(file.read(), maintype="application", subtype="csv", filename=output_filename)

    # Send Email using 163 SMTP (SSL)
    try:
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.163.com", 465, context=context) as server:
            server.login(sender_email, sender_password)
            server.send_message(msg)

        print("\n📧 Email notification sent successfully!")
    except Exception as e:
        print(f"\n⚠️ Failed to send email: {e}")


In [ ]:
# Example usage
recipient_email = "rija.ansari@mail.utoronto.ca"  # Replace with you email account

if df_results.shape == (25,11):
  send_email_notification(recipient_email, output_filename)
